<a href="https://colab.research.google.com/github/JandreiSS/analise-fundamentalista/blob/master/An%C3%A1lise_Fundamentalista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import string
import requests
import warnings
warnings.filterwarnings('ignore')

In [2]:
url = 'http://fundamentus.com.br/resultado.php'

In [3]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [4]:
r = requests.get(url, headers=header)

In [5]:
dflist = pd.read_html(r.text, thousands='.', decimal=',')[0]

In [6]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  dflist[coluna] = dflist[coluna].str.replace('.', '')
  dflist[coluna] = dflist[coluna].str.replace(',', '.')
  dflist[coluna] = dflist[coluna].str.rstrip('%').astype('float') / 100

In [7]:
# dflist['Papel'].values

#<strong>Analisando Dados</strong>

In [8]:
df = dflist[dflist['Liq.2meses'] > 1000000]

<p><strong>Enterprise Value / Earnings Before Interest and Taxes</strong></p>
<p>(Valor do Negócio / Lucro Antes dos Juros e Taxas)</p>

In [9]:
ranking = pd.DataFrame()
ranking['Position'] = range(1,151)
ranking['EV/EBIT'] = df[df['EV/EBIT'] > 0].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values

In [10]:
ranking

,Position,EV/EBIT,ROIC
0,1,PSSA3,WIZS3
1,2,SMLS3,PSSA3
2,3,WIZS3,TOTS3
3,4,ENAT3,ODPV3
4,5,ALUP11,SMLS3
...,...,...,...
145,146,PTBL3,JHSF3
146,147,GNDI3,CAML3
147,148,JHSF3,BRKM5
148,149,PNVL3,LOGG3


Cálculo utilizando o rankeamento pela Fórmula Mágica: Buscar o papel, pegando a posição que ele tem no EV/EBIT e somando a posição que ele tem no ROIC. Quanto menor, melhor

In [11]:
auxA = ranking.pivot_table(columns='EV/EBIT', values='Position')
auxB = ranking.pivot_table(columns='ROIC', values='Position')
t = pd.concat([auxA, auxB])

In [12]:
t

,AALR3,ABEV3,AGRO3,ALPA4,ALSO3,ALUP11,AMAR3,ANIM3,ARZZ3,B3SA3,...,VULC3,VVAR3,WIZS3,WSON33,YDUQ3,EZTC3,HAPV3,LWSA3,MGLU3,WEGE3
Position,114.0,85.0,64.0,136.0,84.0,5.0,50.0,122.0,138.0,139.0,...,52.0,143.0,3.0,25.0,76.0,NaN,NaN,NaN,NaN,NaN
Position,141.0,15.0,84.0,20.0,131.0,29.0,52.0,116.0,30.0,34.0,...,65.0,110.0,1.0,28.0,17.0,117.0,71.0,99.0,107.0,11.0


In [13]:
rank = t.dropna(axis=1).sum().sort_values()
rank[:15]

PSSA3      3.0
WIZS3      4.0
SMLS3      7.0
SAPR4     20.0
SAPR11    22.0
SAPR3     23.0
VALE3     24.0
LEVE3     31.0
ALUP11    34.0
EQTL3     34.0
MRFG3     35.0
BEEF3     37.0
TGMA3     46.0
CVCB3     49.0
ENAT3     53.0
dtype: float64

In [13]:
# Cotação, P/L, EBITDA, Div.Yield, ROE, EV/EBIT
# Comparar papéis do mesmo setor
# lista = df.loc[:, ['Papel', 'Cotação', 'P/L', 'Div.Yield', 'ROE', 'EV/EBIT']]

In [103]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [104]:
def get(url, timeout):
    return requests.get(url, headers=header, timeout = timeout)

In [105]:
def requestUrls(urls, timeout = 5):
    with ThreadPoolExecutor(max_workers = 5) as executor:
        agenda = { executor.submit(get, url, timeout): url for url in urls }

        for tarefa in as_completed(agenda):     
            try:
                conteudo = tarefa.result()
            except Exception as e:
                print ("Não foi possível fazer a requisição! \n{}".format(e))
            else:
                yield conteudo

In [106]:
urls = 'https://www.fundamentus.com.br/detalhes.php?papel=' + lista['Papel'].values
# urls = 'https://www.fundamentus.com.br/detalhes.php?papel=' + df['Papel'].values
# urls = 'https://www.fundamentus.com.br/detalhes.php?papel=' + dflist['Papel'].values

In [107]:
requisicoes = requestUrls(urls, timeout=7)

In [108]:
lista_empresas = []

In [109]:
for requisicao in requisicoes:
    codigo = requisicao.status_code
    url = requisicao.url
    conteudo = requisicao.content
    lista_empresas += [conteudo]
    # print ("{}: {}".format(codigo, url))

In [110]:
html_data = []

In [162]:
# Papel que quer pesquisar
papel = ['WEGE3', 'USIM5']

In [112]:
from bs4 import BeautifulSoup

In [163]:
def readHTML(content):
  soup = BeautifulSoup(content, 'html.parser')
  jsonlist = []
  for tr in soup.find_all('tr'):
    tds = soup.find_all('td')
    papel_html = tds[1].text
    cotacao_html = tds[3].text
    cotacao_html = cotacao_html.replace('.', '')
    cotacao_html = cotacao_html.replace(',', '.')
    cotacao_html = round(float(cotacao_html), 2)
    empresa_html = tds[9].text
    setor_html = tds[13].text
    subsetor_html = tds[17].text
    p_l_html = tds[33].text
    p_l_html = p_l_html.replace('.', '')
    p_l_html = p_l_html.replace(',', '.')
    div_yield_html = tds[75].text
    div_yield_html = div_yield_html.replace('.', '')
    div_yield_html = div_yield_html.replace(',', '.')
    div_yield_html = div_yield_html.replace('%', '')
    div_yield_html = round(float(div_yield_html), 4)
    roe_html = tds[77].text
    roe_html = roe_html.replace('.', '')
    roe_html = roe_html.replace(',', '.')
    roe_html = roe_html.replace('%', '')
    roe_html = round((float(roe_html) / 100), 4)
    valor_firma_html = tds[25].text
    valor_firma_html = valor_firma_html.replace('.', '')
    valor_firma_html = valor_firma_html.replace(',', '.')
    valor_firma_html = round(float(valor_firma_html), 2)
    ev_ebitda_html = tds[81].text
    ev_ebitda_html = ev_ebitda_html.replace('.', '')
    ev_ebitda_html = ev_ebitda_html.replace(',', '.')
    ev_ebitda_html = round(float(ev_ebitda_html), 2)
    ebitda_html = round((valor_firma_html / ev_ebitda_html), 2)

    mid_string = (' \
      "papel": "{}", \
      "cotacao": {}, \
      "empresa": "{}", \
      "setor": "{}", \
      "subsetor": "{}", \
      "p_l": {}, \
      "dividend_yield": {}, \
      "roe": {}, \
      "enterprise_value": {}, \
      "ev_ebitda": {}, \
      "ebitda": {}'.format(papel_html, cotacao_html, empresa_html, setor_html, subsetor_html, p_l_html, div_yield_html, roe_html, valor_firma_html, ev_ebitda_html, ebitda_html))
    
    complete_string = '{ "ticket": {' + mid_string + '} }'
    return complete_string

In [ ]:
import json

In [173]:
for i in range(len(lista_empresas)):
  ticket = pd.read_html(lista_empresas[i])[0][1][0]
  for j in range(len(papel)):
    if ticket == papel[j]:
      conteudo = lista_empresas[i]
      html_data = [conteudo]
      complete_string = readHTML(conteudo)
      print (complete_string)
      j = eval(complete_string)
      # r = requests.post('http://localhost:3333', json=j)
      

{ "ticket": {       "papel": "USIM5",       "cotacao": 7.52,       "empresa": "USIMINAS PNA N1",       "setor": "Siderurgia e Metalurgia",       "subsetor": "Siderurgia",       "p_l": -30.38,       "dividend_yield": 0.6,       "roe": -0.022,       "enterprise_value": 12975900000.0,       "ev_ebitda": 6.08,       "ebitda": 2134194078.95} }
{ "ticket": {       "papel": "WEGE3",       "cotacao": 53.99,       "empresa": "WEG SA ON N1",       "setor": "Máquinas e Equipamentos",       "subsetor": "Motores, Compressores e Outros",       "p_l": 64.83,       "dividend_yield": 0.7,       "roe": 0.181,       "enterprise_value": 112871000000.0,       "ev_ebitda": 41.52,       "ebitda": 2718473025.05} }
